In [1]:
from backbone_cnn import BackboneNet
import torch.nn.functional as F
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [2]:
n_epochs = 3
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=False)

In [4]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

In [5]:
net = BackboneNet()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [6]:
running_loss = 0.0
for epoch in range(10):
    
    for idx, (data, target) in tqdm(enumerate(train_loader)):
        
        optimizer.zero_grad()
        output = net(data)
        loss =  F.nll_loss(output, target) 
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch} loss: {loss.item()}")

0it [00:00, ?it/s]/home/ali/ml/handwriting_data_generator/model/backbone_cnn.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
600it [00:53, 11.14it/s]
1it [00:00,  9.84it/s]

epoch: 0 loss: 0.11841829121112823


600it [00:56, 10.59it/s]
2it [00:00, 10.38it/s]

epoch: 1 loss: 0.0481337308883667


600it [00:53, 11.31it/s]
2it [00:00, 11.35it/s]

...

epoch: 7 loss: 0.03492874279618263


600it [00:52, 11.40it/s]
1it [00:00,  9.88it/s]

epoch: 8 loss: 0.009917395189404488


600it [00:52, 11.41it/s]

epoch: 9 loss: 0.0010756965493783355


In [16]:
loss.item()

0.0010756965493783355

In [17]:
torch.cuda.current_device()

0

In [18]:
net.eval()

BackboneNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3136, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [19]:
def test(mynet):
    for data,target in test_loader:
        x=mynet(data).data.argmax(1).eq(target).sum()
        print(x.item()/len(target))
test(net)

0.991
0.986
0.982
0.991
0.989
0.996
0.995
1.0
0.998
0.996


In [20]:
torch.save(net.state_dict(), 'backbone_net')

In [21]:
new_net = BackboneNet()

In [22]:
test(new_net)

0.108
0.117
0.121
0.101
0.094
0.118
0.128
0.139
0.124
0.139


In [23]:
new_net.load_state_dict(torch.load('backbone_net'))

<All keys matched successfully>

In [24]:
test(new_net)

0.991
0.986
0.982
0.991
0.989
0.996
0.995
1.0
0.998
0.996


In [38]:
net.eval()

BackboneNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3136, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [28]:
net.conv1

Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [29]:
backbone = torchvision.models.mobilenet_v2(pretrained=True).features

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/ali/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:02<00:00, 5.02MB/s]


In [30]:
backbone

Sequential(
  (0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      

In [33]:
backbone = nn.Sequential(net.conv1, net.pool, net.conv2, net.conv2_drop, net.pool)

In [34]:
backbone

Sequential(
  (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (3): Dropout2d(p=0.5, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [35]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [36]:
anchor_generator = AnchorGenerator(sizes=((5, 10, 20, 50),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

In [39]:
backbone.out_channels = 64

In [49]:
backbone(data)

tensor([[[[ 9.3720e-01,  1.6194e+00,  1.4339e+00,  ...,  1.5739e+00,
            4.1414e+00,  1.5996e+00],
          [ 7.1793e-01,  1.3241e+00,  2.0598e+00,  ..., -8.4847e+00,
            3.1875e+00,  1.1550e+00],
          [ 7.1793e-01,  1.4950e+00,  5.5703e+00,  ..., -6.0971e+00,
            2.3716e+00,  5.9476e-01],
          ...,
          [ 7.1793e-01,  1.6378e+00, -5.9613e-01,  ...,  6.3856e-01,
            1.3273e+00,  1.2374e+00],
          [ 7.9171e-01,  2.9952e+00, -1.0352e+01,  ..., -5.6506e-01,
            1.7529e+00,  9.5510e-01],
          [ 1.3247e+00,  4.4653e+00,  5.1468e+00,  ...,  1.3899e+00,
            1.8368e+00,  1.1546e+00]],

         [[-2.2130e-01, -2.7220e+00, -2.8144e+00,  ..., -2.5049e+00,
            3.8725e+00, -1.5358e+00],
          [ 3.7265e-01, -1.5178e+00, -9.6131e-01,  ...,  1.6295e+01,
            1.5519e+01, -1.0102e+00],
          [ 3.7265e-01, -1.4861e+00,  7.0767e+00,  ...,  1.8334e+01,
            1.2726e+01, -1.1912e+00],
          ...,
     